# 뉴럴 네트워크 학습 알고리즘 구현

## Import modules

In [1]:
import tensorflow as tf

## 하이퍼파라미터 정의

In [2]:
EPOCHS = 10

## 네트워크 구조 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.dense1 = tf.keras.layers.Dense(32, activation='relu') #뉴런 갯수 정함
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')       
    
    def call(self, x, training=None, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

## 학습 함수 구현

In [4]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)  # BatchSize x 10(Classes)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

## 테스트 함수 구현

In [5]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)  # BatchSize x 10(Classes)
    loss = loss_object(labels, predictions)
    
    test_loss(loss)
    test_accuracy(labels, predictions)

## 데이터 불러오기

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 모델 생성

In [7]:
model = MyModel()

## 손실함수 및 최적화 알고리즘 정의

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 성능 지표 정의

In [9]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 구현

In [10]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch +1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.28133076429367065, Accuracy: 91.38999938964844, Test Loss: 0.1653043031692505, Test Accuracy: 95.02000427246094
Epoch 2, Loss: 0.1409933865070343, Accuracy: 95.74333190917969, Test Loss: 0.14915834367275238, Test Accuracy: 95.70000457763672
Epoch 3, Loss: 0.1091289073228836, Accuracy: 96.61833190917969, Test Loss: 0.12123986333608627, Test Accuracy: 96.62000274658203
Epoch 4, Loss: 0.08919555693864822, Accuracy: 97.16166687011719, Test Loss: 0.11657709628343582, Test Accuracy: 96.62000274658203
Epoch 5, Loss: 0.0788148045539856, Accuracy: 97.5199966430664, Test Loss: 0.11248157918453217, Test Accuracy: 96.86000061035156
Epoch 6, Loss: 0.06693009287118912, Accuracy: 97.87667083740234, Test Loss: 0.11175088584423065, Test Accuracy: 96.94999694824219
Epoch 7, Loss: 0.06074082851409912, Accuracy: 98.04666900634766, Test Loss: 0.1207413598895073, Test Accuracy: 96.7300033569336
Epoch 8, Loss: 0.05431508272886276, Accuracy: 98.23999786376953, Test Loss: 0.11703226715326309, 